# 评价指标的预处理方法

一般情况下，在综合评价指标中，各指标可能属于不同类型、不同单位或不同数量级，从而使得各指标之间存在着不可公度性。为消除由于各项指标简的这些差别带来的影响，避免出现不合理的评价结果，就需要对评价指标进行一定的预处理，包括对指标的<span style="color: orange">一致化处理和无量纲化处理</span>。

## <span style="color: red">1.指标的一致化处理</span>

即将评价指标的类型进行统一，在评价指标体系中，可能会同时存在<span style="color: orange">极大型指标、极小型指标、居中型指标和区间型指标</span>，他们都具有不同的特点。若指标体系中存在不同类型的指标，必须在综合评价之前将评价指标的类型做一致化处理

#### <span style="color: orange">1.极小型指标化为极大型指标</span>

只需对指标$x_j$取倒数：

$x'_j = \frac{1}{x_j}$

或做平移变换：

$x'_j = M_j-x_j$

其中$M_j = max_{1 \leq i \leq n}\{a_{ij}\}$，即$n$个评价对象第$j$项指标值$a_{ij}$最大值

In [1]:
def min_to_max_reciprocal(min_series):
    """取倒数法：适用于严格正数的极小型指标"""
    return 1 / min_series

# 示例
import pandas as pd
data = pd.Series([1, 2, 3, 4, 5])  # 原始极小型指标
transformed = min_to_max_reciprocal(data)
print("取倒数法转换结果:")
print(transformed)

取倒数法转换结果:
0    1.000000
1    0.500000
2    0.333333
3    0.250000
4    0.200000
dtype: float64


综合示例

In [2]:
import pandas as pd

def transform_min_to_max(df, columns, method='reciprocal'):
    """
    将DataFrame中的极小型列转化为极大型列
    
    参数:
    df: 原始DataFrame
    columns: 需要转换的列名列表
    method: 转换方法 ('reciprocal', 'negative', 'subtract', 'normalization')
    
    返回:
    转换后的DataFrame
    """
    df_transformed = df.copy()
    
    for col in columns:
        if method == 'reciprocal':  #取倒数法
            df_transformed[col] = 1 / df[col]
        elif method == 'negative':  #取负数法
            df_transformed[col] = -df[col]
        elif method == 'subtract':  #差值法（减去最大值）
            max_val = df[col].max()
            df_transformed[col] = max_val - df[col]
        elif method == 'normalization':  #标准化转换（Min-Max归一化变体）
            max_val = df[col].max()
            min_val = df[col].min()
            df_transformed[col] = (max_val - df[col]) / (max_val - min_val)
    
    return df_transformed

# 示例DataFrame
data = {
    '成本': [100, 120, 80, 90, 110],  # 极小型指标
    '质量': [8, 7, 9, 8, 7],          # 极大型指标
    '交货时间': [5, 7, 4, 6, 8]        # 极小型指标
}
df = pd.DataFrame(data)

print("原始数据:")
print(df)

# 转换极小型指标（成本和交货时间）
df_transformed = transform_min_to_max(df, ['成本', '交货时间'], method='subtract')

print("\n转换后数据:")
print(df_transformed)

原始数据:
    成本  质量  交货时间
0  100   8     5
1  120   7     7
2   80   9     4
3   90   8     6
4  110   7     8

转换后数据:
   成本  质量  交货时间
0  20   8     3
1   0   7     1
2  40   9     4
3  30   8     2
4  10   7     0


#### <span style="color: orange">2.居中型指标化为极大型指标</span>

常用的转化方法是计算指标值与中间值的绝对差值的相反数：

极大型指标 = -|原始指标 - 中间值|

In [5]:
import numpy as np
import pandas as pd

def central_to_maximal(data, central_value):
    """
    将居中型指标转化为极大型指标
    
    参数:
    data: 原始数据,可以是数值、列表、numpy数组或pandas Series
    central_value: 中间最佳值
    
    返回:
    转化后的极大型指标
    """
    return -np.abs(np.array(data) - central_value)

# 示例数据
original_data = [4, 5, 6, 7, 8, 9, 10]  # 假设最佳中间值是7
central_value = 7

# 转化
maximal_data = central_to_maximal(original_data, central_value)

# 创建DataFrame展示结果
df = pd.DataFrame({
    '原始数据': original_data,
    '转化后数据': maximal_data
})

print("居中型指标转化为极大型指标示例:")
print(df)
print("\n解释: 越接近中间值7的数据,转化后的值越大")

居中型指标转化为极大型指标示例:
   原始数据  转化后数据
0     4     -3
1     5     -2
2     6     -1
3     7      0
4     8     -1
5     9     -2
6    10     -3

解释: 越接近中间值7的数据,转化后的值越大


#### <span style="color: orange">3.区间型指标化为极大型指标</span>

常用的转化方法是使用距离函数，将指标值到最佳区间的距离转化为极大型指标：

设最佳区间为[a, b]

转化公式：

当$x < a$时：$f(x) = 1 - \frac{a - x}{M}$

当$a ≤ x ≤ b$时：$f(x) = 1$

当$x > b$时：$f(x) = 1 - \frac{x - b}{M}$

其中M为最大允许偏离值，通常取M = max(a - min(x), max(x) - b)

In [6]:
import numpy as np

def interval_to_maximal(data, optimal_range, M=None):
    """
    将区间型指标转化为极大型指标
    
    参数:
    data: 原始数据数组
    optimal_range: 最佳区间，格式为(a, b)
    M: 最大允许偏离值,如果为None则自动计算
    
    返回:
    转化后的极大型指标数组
    """
    a, b = optimal_range
    data = np.asarray(data)  #将输入的数据转换为 NumPy 数组
    
    if M is None:
        M = max(a - np.min(data), np.max(data) - b)
    
    transformed = np.zeros_like(data)
    
    # 当x < a时
    mask = data < a
    transformed[mask] = 1 - (a - data[mask]) / M
    
    # 当a ≤ x ≤ b时
    mask = (data >= a) & (data <= b)
    transformed[mask] = 1
    
    # 当x > b时
    mask = data > b
    transformed[mask] = 1 - (data[mask] - b) / M
    
    return transformed

# 示例数据
data = np.array([2, 3, 4, 5, 6, 7, 8, 9, 10])
optimal_range = (4, 7)  # 最佳区间是4-7

# 转换
transformed_data = interval_to_maximal(data, optimal_range)

print("原始数据:", data)
print("转换后数据:", transformed_data)

原始数据: [ 2  3  4  5  6  7  8  9 10]
转换后数据: [0 0 1 1 1 1 0 0 0]


## <span style="color: red">2.指标的无量纲化处理</span>

也被称为指标的规范化，是通过数学变换来消除原始指标的单位及其数值数量级影响的过程，因此，就有指标的实际值和评价值之分。一般地，将指标无量纲化处理以后的值称为指标评价值。

#### <span style="color: orange">1.Min-Max标准化（Min-Max Normalization）</span>是最常用的数据无量纲化方法之一，也称为离差标准化。它将原始数据线性变换到指定的范围内（通常是[0, 1]区间）。

In [10]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler

# 原始数据
data = np.array([[100, 0.5], 
                 [80, 0.4], 
                 [120, 0.6], 
                 [90, 0.55]])

# Min-Max标准化
scaler = MinMaxScaler()
normalized_data = scaler.fit_transform(data)  #按列处理（特征方向）

print("Min-Max标准化结果:")
print(normalized_data)

Min-Max标准化结果:
[[0.5  0.5 ]
 [0.   0.  ]
 [1.   1.  ]
 [0.25 0.75]]


手动实现

X' = (X - X_min) / (X_max - X_min)

In [9]:
import numpy as np

# 原始数据
data = np.array([[100, 0.5], 
                 [80, 0.4], 
                 [120, 0.6], 
                 [90, 0.55]])

def min_max_normalize(data, feature_range=(0, 1)):
    """
    手动实现Min-Max标准化
    :param data: 二维数组或矩阵
    :param feature_range: 标准化后的范围，默认为(0,1)
    :return: 标准化后的数据
    """
    data = np.array(data)
    min_vals = np.min(data, axis=0)
    max_vals = np.max(data, axis=0)
    range_diff = max_vals - min_vals
    
    # 处理常数列（避免除以0）
    range_diff[range_diff == 0] = 1
    
    # 标准化到[0,1]
    normalized = (data - min_vals) / range_diff
    
    # 如果指定了其他范围
    if feature_range != (0, 1):
        new_min, new_max = feature_range
        normalized = normalized * (new_max - new_min) + new_min
    
    return normalized

# 使用自定义函数
manual_normalized = min_max_normalize(data)
print("\n手动实现结果:")
print(manual_normalized)


手动实现结果:
[[0.5  0.5 ]
 [0.   0.  ]
 [1.   1.  ]
 [0.25 0.75]]


方法3：使用Pandas实现

In [18]:
import pandas as pd

# 原始数据
data = np.array([[100, 0.5], 
                 [80, 0.4], 
                 [120, 0.6], 
                 [90, 0.55]])

df = pd.DataFrame(data, columns=['销售额', '利润率'])

# 对每列进行Min-Max标准化
df_normalized = df.apply(lambda x: (x - x.min()) / (x.max() - x.min()))

print("\nPandas实现结果:")
print(df_normalized)


Pandas实现结果:
    销售额   利润率
0  0.50  0.50
1  0.00  0.00
2  1.00  1.00
3  0.25  0.75


#### <span style="color: orange">2.Z-score标准化（又称标准差标准化）</span>是另一种常用的数据无量纲化方法，它将数据转换为均值为0、标准差为1的分布。

X' = (X - μ) / σ

其中：

X' 是标准化后的值

X 是原始值

μ 是该特征的均值

σ 是该特征的标准差

##### 方法1：使用scikit-learn的StandardScaler

In [11]:
from sklearn.preprocessing import StandardScaler
import numpy as np

# 原始数据
data = np.array([[100, 0.5], 
                 [80, 0.4], 
                 [120, 0.6], 
                 [90, 0.55]])

# 创建StandardScaler对象
scaler = StandardScaler()

# 拟合数据并转换
standardized_data = scaler.fit_transform(data)

print("标准化后的数据:")
print(standardized_data)

print("\n各特征的均值:", scaler.mean_)
print("各特征的标准差:", scaler.scale_)  # 注意：scale_存储的是标准差

标准化后的数据:
[[ 0.16903085 -0.16903085]
 [-1.18321596 -1.52127766]
 [ 1.52127766  1.18321596]
 [-0.50709255  0.50709255]]

各特征的均值: [97.5     0.5125]
各特征的标准差: [14.79019946  0.073951  ]


##### 方法2：手动实现

In [12]:
import numpy as np

# 原始数据
data = np.array([[100, 0.5], 
                 [80, 0.4], 
                 [120, 0.6], 
                 [90, 0.55]])

def z_score_normalize(data):
    """
    手动实现Z-score标准化
    :param data: 二维数组或矩阵
    :return: 标准化后的数据
    """
    data = np.array(data)
    mean = np.mean(data, axis=0)
    std = np.std(data, axis=0)
    
    # 处理常数列（避免除以0）
    std[std == 0] = 1
    
    normalized = (data - mean) / std
    return normalized

# 使用自定义函数
manual_standardized = z_score_normalize(data)
print("\n手动实现结果:")
print(manual_standardized)


手动实现结果:
[[ 0.16903085 -0.16903085]
 [-1.18321596 -1.52127766]
 [ 1.52127766  1.18321596]
 [-0.50709255  0.50709255]]


方法3：使用Pandas实现

In [19]:
import pandas as pd

# 原始数据
data = np.array([[100, 0.5], 
                 [80, 0.4], 
                 [120, 0.6], 
                 [90, 0.55]])

df = pd.DataFrame(data, columns=['销售额', '利润率'])

# 对每列进行Z-score标准化
df_standardized = df.apply(lambda x: (x - x.mean()) / x.std())

print("\nPandas实现结果:")
print(df_standardized)


Pandas实现结果:
        销售额       利润率
0  0.146385 -0.146385
1 -1.024695 -1.317465
2  1.317465  1.024695
3 -0.439155  0.439155


##### <span style="color: orange">3.小数定标标准化（Decimal Scaling）</span>是一种简单的数据标准化方法，它通过移动数据的小数点位置来实现无量纲化。

X' = X / (10^j)

其中：

X' 是标准化后的值

X 是原始值

j 是使得所有值的绝对值都小于1的最小整数（即满足 max(|X'|) < 1 的最小j值）

方法1：基础实现

In [13]:
import numpy as np

def decimal_scaling(data):
    """
    小数定标标准化
    :param data: 数值型数组或矩阵
    :return: 标准化后的数据
    """
    data = np.array(data)
    # 计算每列所需的j值
    j = np.ceil(np.log10(np.max(np.abs(data), axis=0)))
    # 标准化
    scaled_data = data / (10 ** j)
    return scaled_data

# 示例数据
data = np.array([[5300, 0.15], 
                 [4800, 0.12], 
                 [6100, 0.18], 
                 [4900, 0.14]])

# 应用小数定标标准化
scaled_data = decimal_scaling(data)

print("原始数据:")
print(data)
print("\n小数定标标准化结果:")
print(scaled_data)

原始数据:
[[5.3e+03 1.5e-01]
 [4.8e+03 1.2e-01]
 [6.1e+03 1.8e-01]
 [4.9e+03 1.4e-01]]

小数定标标准化结果:
[[0.53 0.15]
 [0.48 0.12]
 [0.61 0.18]
 [0.49 0.14]]


方法2：Pandas实现

In [14]:
import pandas as pd

def decimal_scaling_pandas(df):
    """
    Pandas实现的小数定标标准化
    :param df: Pandas DataFrame
    :return: 标准化后的DataFrame
    """
    # 计算每列的j值
    j = np.ceil(np.log10(df.abs().max()))
    # 标准化
    return df.div(10**j)

# 创建DataFrame
df = pd.DataFrame(data, columns=['销售额', '利润率'])

# 应用标准化
df_scaled = decimal_scaling_pandas(df)

print("\nPandas实现结果:")
print(df_scaled)


Pandas实现结果:
    销售额   利润率
0  0.53  0.15
1  0.48  0.12
2  0.61  0.18
3  0.49  0.14


##### <span style="color: orange">4.向量归一化（也称为范数归一化或单位向量归一化）</span>是一种将样本向量转换为单位向量的标准化方法，使向量的范数（长度）变为1。

X' = X / ||X||

其中：

X' 是归一化后的向量

X 是原始向量

||X|| 是向量的范数（默认为L2范数，即欧几里得距离）

L2范数计算公式：

$||X||₂ = \sqrt{x₁² + x₂² + ... + xₙ²}$

方法1：使用scikit-learn的Normalizer

In [15]:
from sklearn.preprocessing import Normalizer
import numpy as np

# 原始数据（每行是一个向量）
data = np.array([[4, 1, 2, 2],
                 [1, 3, 9, 3],
                 [5, 7, 5, 1]])

# 创建Normalizer对象（默认l2范数）
normalizer = Normalizer()

# 向量归一化
normalized_data = normalizer.fit_transform(data)

print("向量归一化结果:")
print(normalized_data)

# 验证范数是否为1
norms = np.linalg.norm(normalized_data, axis=1)
print("\n归一化后向量的范数:", norms)

向量归一化结果:
[[0.8 0.2 0.4 0.4]
 [0.1 0.3 0.9 0.3]
 [0.5 0.7 0.5 0.1]]

归一化后向量的范数: [1. 1. 1.]


方法2：手动计算L2范数归一化

In [16]:
import numpy as np

# 原始数据（每行是一个向量）
data = np.array([[4, 1, 2, 2],
                 [1, 3, 9, 3],
                 [5, 7, 5, 1]])

def vector_normalize_l2(data):
    """
    手动实现L2范数归一化
    :param data: 二维数组，每行是一个向量
    :return: 归一化后的数组
    """
    norms = np.linalg.norm(data, ord=2, axis=1, keepdims=True)
    # 处理零范数（避免除以0）
    norms[norms == 0] = 1
    return data / norms

# 手动归一化
manual_normalized = vector_normalize_l2(data)

print("\n手动实现结果:")
print(manual_normalized)


手动实现结果:
[[0.8 0.2 0.4 0.4]
 [0.1 0.3 0.9 0.3]
 [0.5 0.7 0.5 0.1]]


方法3：使用Pandas实现

In [17]:
import pandas as pd

# 原始数据（每行是一个向量）
data = np.array([[4, 1, 2, 2],
                 [1, 3, 9, 3],
                 [5, 7, 5, 1]])

# 创建DataFrame
df = pd.DataFrame(data, columns=['f1', 'f2', 'f3', 'f4'])

# 向量归一化
df_normalized = df.div(np.linalg.norm(df.values, axis=1), axis=0)

print("\nPandas实现结果:")
print(df_normalized)


Pandas实现结果:
    f1   f2   f3   f4
0  0.8  0.2  0.4  0.4
1  0.1  0.3  0.9  0.3
2  0.5  0.7  0.5  0.1


##### <span style="color: orange">5.鲁棒标准化（Robust Scaling）</span>是一种对异常值不敏感的数据标准化方法，它使用中位数和四分位距（IQR）进行缩放，而不是均值和标准差。

X' = (X - median) / IQR

其中：

X' 是标准化后的值

X 是原始值

median 是该特征的中位数

IQR 是四分位距（第75百分位数 - 第25百分位数）

方法1：使用scikit-learn的RobustScaler

In [20]:
from sklearn.preprocessing import RobustScaler
import numpy as np

# 包含异常值的数据
data = np.array([[1.0, 10.0],
                 [2.0, 15.0],
                 [3.0, 20.0],
                 [4.0, 25.0],
                 [5.0, 50.0]])  # 最后一个值是异常值

# 创建RobustScaler对象
scaler = RobustScaler()

# 拟合数据并转换
robust_data = scaler.fit_transform(data)

print("鲁棒标准化结果:")
print(robust_data)

print("\n中位数:", scaler.center_)
print("四分位距:", scaler.scale_)  # scale_存储的是IQR

鲁棒标准化结果:
[[-1.  -1. ]
 [-0.5 -0.5]
 [ 0.   0. ]
 [ 0.5  0.5]
 [ 1.   3. ]]

中位数: [ 3. 20.]
四分位距: [ 2. 10.]


方法2：手动实现

In [21]:
import numpy as np

# 包含异常值的数据
data = np.array([[1.0, 10.0],
                 [2.0, 15.0],
                 [3.0, 20.0],
                 [4.0, 25.0],
                 [5.0, 50.0]])  # 最后一个值是异常值

def robust_scaling(data):
    """
    手动实现鲁棒标准化
    :param data: 二维数组或矩阵
    :return: 标准化后的数据
    """
    data = np.array(data)
    median = np.median(data, axis=0)
    q75 = np.percentile(data, 75, axis=0)
    q25 = np.percentile(data, 25, axis=0)
    iqr = q75 - q25
    
    # 处理IQR为0的情况（避免除以0）
    iqr[iqr == 0] = 1
    
    return (data - median) / iqr

# 手动实现
manual_robust = robust_scaling(data)

print("\n手动实现结果:")
print(manual_robust)


手动实现结果:
[[-1.  -1. ]
 [-0.5 -0.5]
 [ 0.   0. ]
 [ 0.5  0.5]
 [ 1.   3. ]]


方法3：使用Pandas实现

In [22]:
import pandas as pd

# 包含异常值的数据
data = np.array([[1.0, 10.0],
                 [2.0, 15.0],
                 [3.0, 20.0],
                 [4.0, 25.0],
                 [5.0, 50.0]])  # 最后一个值是异常值

# 创建DataFrame
df = pd.DataFrame(data, columns=['特征1', '特征2'])

# 计算中位数和IQR
median = df.median()
q75 = df.quantile(0.75)
q25 = df.quantile(0.25)
iqr = q75 - q25

# 鲁棒标准化
df_robust = (df - median) / iqr

print("\nPandas实现结果:")
print(df_robust)


Pandas实现结果:
   特征1  特征2
0 -1.0 -1.0
1 -0.5 -0.5
2  0.0  0.0
3  0.5  0.5
4  1.0  3.0
